# Explorador de Datos y Detección de Anomalías en Series de Tiempo

Este notebook demuestra el uso de la clase `TimeSeriesAnomalyDetector` para analizar series de tiempo y detectar anomalías utilizando diferentes métodos.

## Contenido:
1. Carga y preparación de datos
2. Inicialización del detector de anomalías
3. Aplicación de métodos de detección
4. Visualización de resultados

In [3]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from anomaly_detection import TimeSeriesAnomalyDetector
import warnings
warnings.filterwarnings('ignore')

## 1. Carga y Exploración de Datos

Cargamos la serie de tiempo desde el archivo CSV y realizamos una exploración inicial.

In [8]:
df_d = pd.read_csv('csv_to_dash/time_series.csv')
print("Primeras filas del dataset:")
df_d.head()

Primeras filas del dataset:


,WebId,Id,Name,Path,Descriptor,PointClass,PointType,DigitalSetName,EngineeringUnits,Span,Zero,Step,Future,DisplayDigits,Links,Timestamp,Value
0,F1DPiCe__0Mwb0W8oKjXyO-3UgadoAAAU0xJQlJBUElcUE...,55913,POCO_MRO_002_LP_ESTADO,\\slibrapi\POCO_MRO_002_LP_ESTADO,Estado da Linha de Produção do Poço (Aberto/Fe...,classic,Digital,ESTADO,NaN,1.0,5.0,True,False,0,{'Self': 'https://piwebapicorp.petrobras.com.b...,2024-10-18T15:13:29.0008392Z,NaN
1,F1DPiCe__0Mwb0W8oKjXyO-3UgadoAAAU0xJQlJBUElcUE...,55913,POCO_MRO_002_LP_ESTADO,\\slibrapi\POCO_MRO_002_LP_ESTADO,Estado da Linha de Produção do Poço (Aberto/Fe...,classic,Digital,ESTADO,NaN,1.0,5.0,True,False,0,{'Self': 'https://piwebapicorp.petrobras.com.b...,2024-10-18T15:21:34.4420623Z,NaN


In [ ]:
df_d_filtered = df_d[['WebId','Id','Name','Timestamp', 'Value']]
df_d_filtered.head()


,WebId,Name,Id,Timestamp,Value
0,F1DPiCe__0Mwb0W8oKjXyO-3UgadoAAAU0xJQlJBUElcUE...,POCO_MRO_002_LP_ESTADO,55913,2024-10-18T15:13:29.0008392Z,NaN
1,F1DPiCe__0Mwb0W8oKjXyO-3UgadoAAAU0xJQlJBUElcUE...,POCO_MRO_002_LP_ESTADO,55913,2024-10-18T15:21:34.4420623Z,NaN


In [33]:
# Cargar datos del CSV
df = pd.read_csv('csv/time-series.csv')
print("Primeras filas del dataset:")
df.head()

Primeras filas del dataset:


,Timestamp,Value
0,2025-09-25T20:08:13.7797393Z,61.169
1,2025-09-25T20:09:33.7831726Z,61.17
2,2025-09-25T20:09:38.8496093Z,61.169
3,2025-09-25T20:09:48.8062286Z,61.168
4,2025-09-25T20:10:08.8138275Z,61.169


In [34]:
# Información general del dataset
print("\nInformación del dataset:")
df.info()

print("\nEstadísticas descriptivas:")
df.describe()


Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5999 entries, 0 to 5998
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Timestamp  5999 non-null   object
 1   Value      5999 non-null   object
dtypes: object(2)
memory usage: 93.9+ KB

Estadísticas descriptivas:


,Timestamp,Value
count,5999,5999
unique,5999,2902
top,2025-09-25T20:08:13.7797393Z,61.149
freq,1,34


In [35]:
# Preparar el DataFrame para el análisis
# Convertir Timestamp a datetime y establecer como índice
df_prepared = df.copy()
df_prepared['Timestamp'] = pd.to_datetime(df_prepared['Timestamp'])
df_prepared.set_index('Timestamp', inplace=True)
df_prepared['Value'] = pd.to_numeric(df_prepared['Value'], errors='coerce')

print("Datos preparados (la limpieza se hace automáticamente en add_series):")
df_prepared.head()

Datos preparados (la limpieza se hace automáticamente en add_series):


,Value
Timestamp,
2025-09-25 20:08:13.779739300+00:00,61.169
2025-09-25 20:09:33.783172600+00:00,61.170
2025-09-25 20:09:38.849609300+00:00,61.169
2025-09-25 20:09:48.806228600+00:00,61.168
2025-09-25 20:10:08.813827500+00:00,61.169


## 2. Inicialización del Detector de Anomalías

Creamos una instancia de `TimeSeriesAnomalyDetector` y agregamos nuestra serie de tiempo.

In [36]:
# Inicializar el detector
detector = TimeSeriesAnomalyDetector()

# Agregar la serie de tiempo (la limpieza se hace automáticamente)
detector.add_series('SeriePrincipal', df_prepared)

print("Serie agregada exitosamente.")
print(f"Series disponibles: {list(detector.dataframes.keys())}")
print(f"Forma de los datos limpios: {detector.dataframes['SeriePrincipal'].shape}")

Serie agregada exitosamente.
Series disponibles: ['SeriePrincipal']
Forma de los datos limpios: (5475, 1)


## 3. Aplicación de Métodos de Detección de Anomalías

Aplicamos el método Isolation Forest para detectar anomalías en la serie de tiempo.

In [37]:
# Aplicar Isolation Forest
detector.apply_isolation_forest(
    series_name='SeriePrincipal',
    target_col='Value',
    n_estimators=100,
    contamination=0.01  # Esperamos 1% de anomalías
)

print("Isolation Forest aplicado exitosamente.")

Isolation Forest aplicado exitosamente.


In [38]:
# Verificar resultados
results = detector.results['SeriePrincipal']
print("Resultados de la detección:")
print(results.head())

# Contar anomalías detectadas
anomalies_count = (results['is_anomaly_IF'] == -1).sum()
total_points = len(results)
print(f"\nAnomalías detectadas: {anomalies_count} de {total_points} puntos ({anomalies_count/total_points*100:.2f}%)")

Resultados de la detección:
                                     is_anomaly_IF  anomaly_score_IF
Timestamp                                                           
2025-09-25 20:08:13.779739300+00:00            NaN               NaN
2025-09-25 20:09:33.783172600+00:00            NaN               NaN
2025-09-25 20:09:38.849609300+00:00            NaN               NaN
2025-09-25 20:09:48.806228600+00:00            1.0         -0.507099
2025-09-25 20:10:08.813827500+00:00            1.0         -0.510754

Anomalías detectadas: 55 de 5475 puntos (1.00%)


## 4. Visualización de Resultados

Creamos una visualización interactiva que muestra la serie de tiempo y las anomalías detectadas.

In [39]:
# Crear visualización
fig = detector.plot_anomalies(
    series_name='SeriePrincipal',
    target_col='Value',
    methods_to_plot=['IF']  # IF = Isolation Forest
)

# Mostrar el gráfico
fig.show()

## 5. Análisis Adicional

Exploramos más detalles sobre las anomalías detectadas.

In [40]:
# Obtener las anomalías detectadas
anomalies = results[results['is_anomaly_IF'] == -1]
print("Anomalías detectadas (primeras 10):")
print(anomalies.head(10))

# Estadísticas de las anomalías
if len(anomalies) > 0:
    print("\nEstadísticas de las anomalías:")
    anomaly_values = df_prepared.loc[anomalies.index, 'Value']
    print(anomaly_values.describe())
    
    # Comparar con valores normales
    normal_values = df_prepared.loc[results[results['is_anomaly_IF'] == 1].index, 'Value']
    print("\nEstadísticas de valores normales:")
    print(normal_values.describe())

Anomalías detectadas (primeras 10):
                                     is_anomaly_IF  anomaly_score_IF
Timestamp                                                           
2025-09-27 15:05:25.376327500+00:00           -1.0         -0.696919
2025-09-27 15:06:05.362014700+00:00           -1.0         -0.697863
2025-09-27 15:06:10.360122600+00:00           -1.0         -0.697863
2025-09-27 15:06:15.367156900+00:00           -1.0         -0.700228
2025-09-27 15:06:20.383193900+00:00           -1.0         -0.699754
2025-09-27 15:06:25.354324300+00:00           -1.0         -0.699754
2025-09-27 15:06:30.399581900+00:00           -1.0         -0.702394
2025-09-27 15:06:35.368606500+00:00           -1.0         -0.701919
2025-09-27 15:06:45.383422800+00:00           -1.0         -0.701919
2025-09-27 15:06:50.384490900+00:00           -1.0         -0.702870

Estadísticas de las anomalías:
count    55.000000
mean     63.191527
std       0.291359
min      62.663000
25%      62.965500
50%      

## 6. Extensión a Múltiples Series (Ejemplo)

Demostramos cómo agregar múltiples series de tiempo al detector.

In [41]:
# Crear una segunda serie con algunos valores atípicos para demostración
df2 = df_prepared.copy()
df2['Value'] = df2['Value'] * 3 # Modificar ligeramente

# Agregar algunos valores atípicos
outlier_indices = df2.sample(frac=0.02).index  # 2% de outliers
df2.loc[outlier_indices, 'Value'] = df2.loc[outlier_indices, 'Value'] * 4

# Agregar la segunda serie
detector.add_series('SerieModificada', df2)

print("Segunda serie agregada.")
print(f"Series disponibles: {list(detector.dataframes.keys())}")

Segunda serie agregada.
Series disponibles: ['SeriePrincipal', 'SerieModificada']


In [42]:
# Aplicar análisis a la segunda serie
detector.apply_isolation_forest(
    series_name='SerieModificada',
    target_col='Value',
    contamination=0.02  # Más contaminación esperada
)

# Visualizar comparación
fig2 = detector.plot_anomalies(
    series_name='SerieModificada',
    target_col='Value',
    methods_to_plot=['IF']
)

fig2.show()

## Conclusiones

Este notebook ha demostrado:

1. **Carga y preparación** de datos de series de tiempo
2. **Uso de la clase TimeSeriesAnomalyDetector** para análisis flexible
3. **Aplicación de Isolation Forest** para detección de anomalías
4. **Visualización interactiva** con Plotly
5. **Extensibilidad** para múltiples series y métodos futuros

La clase está diseñada para ser fácilmente extensible con nuevos métodos de detección de anomalías como One-Class SVM, Hampel Filter, etc.